<h1>Faithfulness metrics</h1>

<h2>Introduction</h2>

<p>This notebook allow to retrieve the faithfulness metrics for compared saliency methods</p>
<p>It requires the installation of requirements.txt and jupyter</p>

<h2>Parameters</h2>

In [ ]:
model = 'vgg16'
csv_folder = './csv'

### All available methods by default
### Use the desired subpart
#saliency_dict = {'gradcam':'GradCAM', 'gradcampp':'GradCAM++', 'smoothgradcampp':'SmoothGradCAM++', 
#                 'scorecam':'ScoreCAM',  'sscam':'SSCAM', 'iscam':'ISCAM', 
#                 'zoomcam':'ZoomCAM', 'layercamfusion4':'LayerCAM',
#                 'ig':'IntegratedGradient', 'ixg':'InputXGrad', 'sg':'SmoothGrad',
#                 'rise':'RISE', 'occlusion':'Occlusion',
#                 'pcamp4':'PCAM+', "pcamm4":"PCAM-", 'pcampm4':'PCAM+/-'}

# Example of subsample for CAM methods
saliency_dict = {'gradcam':'GradCAM', 'gradcampp':'GradCAM++', 'smoothgradcampp':'SmoothGradCAM++', 
                 'scorecam':'ScoreCAM',  'sscam':'SSCAM', 'iscam':'ISCAM',
                 'zoomcam':'ZoomCAM', 'layercamfusion4':'LayerCAM',
                 'pcampm4':'PCAM+/-'}



In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

In [ ]:
x = (np.array(range(224))+1)/224

<h2>Metrics</h2>

In [ ]:
print(model, "faithfulness metrics")
print('____________________________________')
for saliency in saliency_dict.keys():
    del_auc = pd.read_csv (csv_folder + "/" + "del_auc_" + model + "_" + saliency + ".csv").to_numpy()
    ins_auc = pd.read_csv (csv_folder + "/" + "ins_auc_" + model + "_" + saliency + ".csv").to_numpy()
    
    del_details = pd.read_csv (csv_folder + "/" + "del_details_" + model + "_" + saliency + ".csv").to_numpy()
    del_details_float = np.array(del_details[:, 1:], dtype=float)
    mean_del_auc = np.mean(del_auc[:,1])
    
    ins_details = pd.read_csv (csv_folder + "/" + "ins_details_" + model + "_" + saliency + ".csv").to_numpy()
    ins_details_float = np.array(ins_details[:, 1:], dtype=float)
    mean_ins_auc = np.mean(ins_auc[:,1])
    
    try:
        sens = pd.read_csv (csv_folder + "/" + "sens_" + model + "_" + saliency + ".csv").to_numpy()
        mean_sens = np.mean(sens[:,1])
    except:
        mean_sens = 0

    print(saliency_dict[saliency])
    print('Mean Insertion auc:           %0.3f' % mean_ins_auc)
    print('Mean Deletion  auc:           %0.3f' % mean_del_auc)
    print('Mean Insertion-Deletion auc:  %0.3f' % (mean_ins_auc - mean_del_auc))
    print('Mean Insertion-Deletion auc:  %0.3f' % (mean_ins_auc - mean_del_auc))
    print('Mean Sensitivity:             %0.3f' % mean_sens)
    print('____________________________________')



<h2>Curves</h2>

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16,6))
fontsize = 22
ax[0].set_title("Insertion", fontsize=fontsize)
ax[1].set_title("Deletion", fontsize=fontsize)
ax[2].set_title("Insertion-Deletion", fontsize=fontsize)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14) 
for idx_saliency, saliency in enumerate(saliency_dict.keys()):
    ins_details = pd.read_csv (csv_folder + "/" + "ins_details_" + model + "_" + saliency + ".csv").to_numpy()
    mean_ins_details = ins_details[:, 1:].mean(0)
    ax[0].plot(x, mean_ins_details, label = saliency_dict[saliency], lw=2)
    del_details = pd.read_csv (csv_folder + "/" + "del_details_" + model + "_" + saliency + ".csv").to_numpy()
    mean_del_details = del_details[:, 1:].mean(0)
    ax[1].plot(x, mean_del_details, label = saliency_dict[saliency], lw=2)
    mean_del_details = del_details[:, 1:].mean(0)
    mean_ins_details = ins_details[:, 1:].mean(0)
    ins_del = mean_ins_details - mean_del_details
    ax[2].plot(x, ins_del, label = saliency_dict[saliency], lw=2)
    for cur_ax in ax:
        cur_ax.legend(prop={'size': 16})

plt.suptitle('Metrics on ' + str(ins_details.shape[0]) + ' images ', fontsize = fontsize*1.1)
plt.tight_layout()
display(fig)
plt.close()

<h2>Specific image curves</h2>

<p>Allow to generate insertion and deletion curves for any individual image.
The number of the image is the index number in the images.txt list, same used in interactive_saliency_visualization.ipynb that can be used simultaly to provide visual feedback on image selection</p>

In [ ]:
# Select image number here
i = 1000

fig, ax = plt.subplots(1, 3, figsize=(16,6))
fontsize = 22
ax[0].set_title("Insertion", fontsize=fontsize)
ax[1].set_title("Deletion", fontsize=fontsize)
ax[2].set_title("Insertion-Deletion", fontsize=fontsize)

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14) 
for idx_saliency, saliency in enumerate(saliency_dict.keys()):
    ins_details = pd.read_csv (csv_folder + "/" + "ins_details_" + model + "_" + saliency + ".csv").to_numpy()
    cur_ins_details = ins_details[i, 1:]
    ax[0].plot(x, cur_ins_details, label = saliency_dict[saliency], lw=2)
    del_details = pd.read_csv (csv_folder + "/" + "del_details_" + model + "_" + saliency + ".csv").to_numpy()
    cur_del_details = del_details[i, 1:]
    ax[1].plot(x, cur_del_details, label = saliency_dict[saliency], lw=2)
    ins_del = cur_ins_details - cur_del_details
    ax[2].plot(x, ins_del, label = saliency_dict[saliency], lw=2)
    for cur_ax in ax:
        cur_ax.legend(prop={'size': 16})
plt.suptitle('Metrics on image id ' + str(i), fontsize = fontsize*1.1)
plt.tight_layout()
display(fig)
plt.close()